**Task-1: Stack Class**

In [5]:
class Stack:
    def __init__(self):
        """Initialize internal storage for the stack using a Python list."""
        self._data = []

    def push(self, item):
        """Place a new item on the top of the stack."""
        self._data.append(item)

    def pop(self):
        """Remove and return the top item from the stack."""
        if self.is_empty():
            raise IndexError("Stack underflow")
        return self._data.pop()

    def peek(self):
        """Return the top item without removing it."""
        if self.is_empty():
            raise IndexError("Empty stack")
        return self._data[-1]

    def is_empty(self):
        """Return True if the stack has no elements."""
        return len(self._data) == 0

    def size(self):
        """Return the current number of elements in the stack."""
        return len(self._data)


# --- Quick tests for Task-1 ---
s = Stack()
assert s.is_empty()
s.push(10)
s.push(20)
assert s.peek() == 20
assert s.pop() == 20
assert s.pop() == 10
try:
    s.pop()
    assert False, "Expected IndexError for underflow"
except IndexError:
    pass
print("Task-1: Stack OK")

Task-1: Stack OK


**Task-2: BracketChecker**

In [2]:
class BracketChecker:
    def __init__(self):
        """Create a new bracket checker that uses an internal Stack instance."""
        self.stack = Stack()  # composition: use a Stack inside

    def is_open(self, ch):
        """Return True if character is an opening bracket."""
        return ch in "([{"

    def _matches(self, open_br, close_br):
        """Return True if open_br correctly matches close_br."""
        pairs = {')': '(', ']': '[', '}': '{'}  # mapping closing -> opening
        return pairs.get(close_br) == open_br

    def is_balanced(self, expr: str) -> bool:
        """Return True if expr has balanced (), [], {}; False otherwise.
        Ignores non-bracket characters.
        """
        # ---
        self.stack = Stack()  # reset stack per call
        for ch in expr:
            if self.is_open(ch):
                self.stack.push(ch)  # push openings
            elif ch in ")]}":
                if self.stack.is_empty():  # unmatched closing bracket
                    return False
                top = self.stack.pop()  # get last opening
                if not self._matches(top, ch):  # mismatch pair
                    return False
            # ignore other characters
        return self.stack.is_empty()  # balanced only if nothing left


# --- Quick tests for Task-2 ---
# Assuming 'Stack' class is defined elsewhere and works correctly (e.g., push, pop, is_empty).
# If Stack is not defined, these tests will fail.

bc = BracketChecker()
assert bc.is_balanced("([{}])") is True       # perfectly nested
assert bc.is_balanced("(({]})") is False      # crossing pairs
assert bc.is_balanced("((()))") is True       # deep nesting
assert bc.is_balanced("(") is False           # close before open (implies an error in the comment, but the code result for "(" is False)
print("Task-2: BracketChecker OK")

Task-2: BracketChecker OK


**Task-3: Infix→Postfix Converter**

In [4]:
# --- 1. Required Stack Class ---
class Stack:
    """A basic Stack implementation using a Python list."""
    def __init__(self):
        self._items = []

    def is_empty(self):
        return not self._items

    def push(self, item):
        self._items.append(item)

    def pop(self):
        if self.is_empty():
            raise IndexError("pop from empty stack")
        return self._items.pop()

    def peek(self):
        if self.is_empty():
            raise IndexError("peek from empty stack")
        return self._items[-1]

# ----------------------------------------------------------------------
# --- 2. Corrected InfixToPostfix Class ---
class InfixToPostfix:
    def __init__(self):
        """Initialize precedence and the internal operator stack."""
        # FIX: Added '^': 4 (or a higher value) to the precedence map
        self._prec = {'*': 3, '/': 3, '+': 2, '-': 2, '^': 4, '(': 1}  # Corrected precedence map
        self._right_assoc = {'^'}                             # right-associative operators
        self._stack = Stack()                                 # operator stack

    def _is_operand(self, ch):
        """Return True if ch is an operand (letter or digit)."""
        return ch.isalnum()

    def convert(self, infix: str) -> str:
        """Convert an infix expression (single-token chars) to postfix (RPN)."""
        out = []  # output token list

        self._stack = Stack()  # reset operator stack
        # remove spaces so we can scan char by char
        for ch in infix.replace(" ", ""):
            if self._is_operand(ch):
                out.append(ch)  # operands go straight to output
            elif ch == '(':
                self._stack.push(ch)  # push opening parenthesis
            elif ch == ')':
                # pop operators until '(' is found
                while not self._stack.is_empty() and self._stack.peek() != '(':
                    out.append(self._stack.pop())
                if self._stack.is_empty():
                    raise ValueError("Mismatched parentheses")  # no matching '('
                self._stack.pop()  # discard '('

            else:  # operator case: pop while stack top has higher precedence
                # or equal precedence for left-associative operators
                while (not self._stack.is_empty()
                       and self._stack.peek() != '('
                       and (self._prec[self._stack.peek()] > self._prec[ch]
                            or (self._prec[self._stack.peek()] == self._prec[ch]
                                and ch not in self._right_assoc))):
                    out.append(self._stack.pop())
                self._stack.push(ch)  # finally push current operator

        # flush remaining operators
        while not self._stack.is_empty():
            top = self._stack.pop()
            if top == '(':
                raise ValueError("Mismatched parentheses")  # stray '('
            out.append(top)

        return "".join(out)  # join tokens to make postfix string

# ----------------------------------------------------------------------
# --- 3. Quick tests for Task-3 ---

conv = InfixToPostfix()
assert conv.convert("(A+B)*C") == "AB+C*"
assert conv.convert("A*(B+C)") == "ABC+*"
assert conv.convert("A+B*C") == "ABC*+"
# This test now passes due to the fix in __init__
assert conv.convert("A^B^C+D") == "ABC^^D+"  # right associative '^'
print("Task-3: InfixToPostfix OK")

Task-3: InfixToPostfix OK


**Task-4: PostfixEvaluator**

In [9]:
# ======================================================================
# 1. The Required Stack Class
# ======================================================================
class Stack:
    """A basic Stack implementation using a Python list."""
    def __init__(self):
        self._items = []

    def is_empty(self):
        return not self._items

    def push(self, item):
        self._items.append(item)

    def pop(self):
        if self.is_empty():
            raise IndexError("pop from empty stack")
        return self._items.pop()

    def size(self):
        return len(self._items)

# ======================================================================
# 2. PostfixEvaluator Class
# ======================================================================
class PostfixEvaluator:
    def __init__(self):
        """Initialize with an internal stack of numbers."""
        self.stack = Stack()

    def _apply(self, op, b, a):
        """Apply binary operator 'op' to operands a (left) and b (right)."""
        if op == '+': return a + b
        if op == '-': return a - b
        if op == '*': return a * b
        if op == '/': return a / b
        if op == '^': return a ** b
        raise ValueError(f"Unknown operator {op}")

    def evaluate(self, postfix: str) -> float:
        """Evaluate a postfix string containing single-digit operands."""
        self.stack = Stack() # reset stack per call
        for ch in postfix.replace(" ", ""):
            if ch.isdigit():
                self.stack.push(float(ch)) # push numeric value
            elif ch in "+-*/^": # operator
                if self.stack.size() < 2:
                    # This is the line that raised the error
                    raise ValueError("Malformed expression: insufficient operands")
                b = self.stack.pop() # right operand
                a = self.stack.pop() # left operand
                self.stack.push(self._apply(ch, b, a))
            else:
                raise ValueError(f"Bad token {ch}") # reject unknown tokens

        if self.stack.size() != 1:
            raise ValueError("Malformed expression: leftover values")
        return self.stack.pop() # final result

# ----------------------------------------------------------------------
# --- Quick tests for Task-4 (Error fixed in test case) ---
# ----------------------------------------------------------------------
ev = PostfixEvaluator()
# This test was previously fixed for a different error:
assert ev.evaluate("432+*") == 20.0   # (4*(3+2)) = 20
assert ev.evaluate("23+5*") == 25.0   # (2+3)*5 = 25
# FIX: Changed "82/-3" to the correct binary postfix "82/3-"
assert ev.evaluate("82/3-") == 1.0    # (8/2) - 3 = 1
print("Task-4: PostfixEvaluator OK")

Task-4: PostfixEvaluator OK


**Task-5: LinearSearch**

In [10]:
class LinearSearch:
    def __init__(self, data):
        """Store a copy of the data to avoid external mutation side-effects."""
        self.data = list(data)

    def find(self, target):
        """Return index of first occurrence of target; -1 if not found."""
        for i, x in enumerate(self.data):  # scan left to right
            if x == target:                # match?
                return i                   # return index of first match
        return -1                          # not found

# --- Quick tests for Task-5 ---
ls = LinearSearch([10, 30, 20, 50])
assert ls.find(20) == 2  # expected index
assert ls.find(99) == -1 # missing
print("Task-5: LinearSearch OK")

Task-5: LinearSearch OK
